In [1]:
import os
import pandas as pd
import json
from tqdm import tqdm
import matplotlib.pyplot as plt



In [2]:
base_path = "/home/rahuladhi/Desktop/AI/MABe/MABe-mouse-behavior-detection"
train_tracking_path = os.path.join(base_path, 'train_tracking')
train_annotation_path = os.path.join(base_path, 'train_annotation')

print("Tracking path exists: ", os.path.exists(train_tracking_path))
print("Annotation path exists: ", os.path.exists(train_annotation_path))


Tracking path exists:  True
Annotation path exists:  True


In [3]:
tracking_folders = sorted(os.listdir(train_tracking_path))
annotation_folders = sorted(os.listdir(train_annotation_path))

print("Number of tracking folders: ", len(tracking_folders))
print("NUmber of annotation folders: ", len(annotation_folders))

print("Example of tracking folders: ", tracking_folders[:5])
print("Example of annotation folders: ", annotation_folders[:5])

Number of tracking folders:  21
NUmber of annotation folders:  19
Example of tracking folders:  ['AdaptableSnail', 'BoisterousParrot', 'CRIM13', 'CalMS21_supplemental', 'CalMS21_task1']
Example of annotation folders:  ['AdaptableSnail', 'BoisterousParrot', 'CRIM13', 'CalMS21_supplemental', 'CalMS21_task1']


In [4]:
tracking_set = set(tracking_folders)
annotation_set = set(annotation_folders)

print("There in tracking but not in annotation: ", tracking_set - annotation_set)
print("There in annotation but not in tracking: ", annotation_set - tracking_set)

There in tracking but not in annotation:  {'MABe22_movies', 'MABe22_keypoints'}
There in annotation but not in tracking:  set()


In [5]:
true_set = tracking_set & annotation_set
true_folders = sorted(true_set)
len(true_folders)


19

In [19]:
mismatch_folders = []
tracking_files = []
annotation_files = []

for folder in true_folders:
    tracking_files_path = os.path.join(train_tracking_path, folder)
    annotation_files_path = os.path.join(train_annotation_path, folder)


    for f in os.listdir(tracking_files_path):
        if f.endswith(".parquet"):
            tracking_files.append(f)
    for f in os.listdir(annotation_files_path):
        if f.endswith(".parquet"):
            annotation_files.append(f)

    tracking_files_set = set(tracking_files)
    annotation_files_set = set(annotation_files)

    #print("Number of tracking files: ", len(tracking_files))
    #print("Number of annotation files: ", len(annotation_files))


    if len(tracking_files)!=len(annotation_files):
        print("Not matching - ", folder)
        mismatch_folders.append(folder)


Not matching -  PleasantMeerkat
Not matching -  ReflectiveManatee
Not matching -  SparklingTapir
Not matching -  TranquilPanther
Not matching -  UppityFerret


In [20]:
for folder in mismatch_folders:
    print(folder)
    tracking_mismatch_file_path = os.path.join(train_tracking_path, folder)
    annotation_mismatch_file_path = os.path.join(train_annotation_path, folder)

    tracking_mismatch_files = sorted(os.listdir(tracking_mismatch_file_path))
    annotation_mismatch_files = sorted(os.listdir(annotation_mismatch_file_path))

    print("Number of files in tracking: ", len(tracking_mismatch_files))
    print("Number of files in annotation: ", len(annotation_mismatch_files))




PleasantMeerkat
Number of files in tracking:  36
Number of files in annotation:  35
ReflectiveManatee
Number of files in tracking:  20
Number of files in annotation:  20
SparklingTapir
Number of files in tracking:  69
Number of files in annotation:  54
TranquilPanther
Number of files in tracking:  25
Number of files in annotation:  25
UppityFerret
Number of files in tracking:  21
Number of files in annotation:  21
